First, analyze the raw token.
This does not validate the signature, just takes the JSON apart and looks at the contents.

In [ ]:
import base64
import json
import logging
import os
import time

token = os.getenv('ACCESS_TOKEN', None)

if token is None:
    logging.fatal('You have no token.')
    
token_parts = token.split('.')

if len(token_parts) != 3:
    logging.fatal('Your token is malformed.')
    
(token_header, token_payload, token_signature) = token_parts

# If the payload isn't padded to the right amount, just add some extra padding.
# Python complains about lack of padding, but not extra padding
token_dict = json.loads(base64.b64decode(token_payload + '====='))

print("Your name is", token_dict['name'])
print("Your uid is", token_dict['uid'])
print("Your uid number is", token_dict['uidNumber'])
print("The token issuer is", token_dict['iss'])

iat = time.gmtime(int(token_dict['iat']))
print("Your token was issued at UTC:", time.asctime(iat))

exp = time.gmtime(int(token_dict['exp']))
print("Your token expires at UTC:   ", time.asctime(exp))

current_time = time.gmtime()
print("The time is currently UTC:   ", time.asctime(current_time))

if current_time < iat:
    logging.error("Your token isn't valid yet.")
    
if iat < current_time and current_time < exp:
    print("Your token is VALID.")

if current_time > exp:
    logging.error("Your token is expired!")
    raise RuntimeError(f"Your token expired at UTC: {time.asctime(exp)}")

Now, try to gather some additional information from the token issuer, including verifying the token.

In [ ]:
import json
from urllib.parse import urljoin

import requests

# Only run on int as that's the only place the analyze endpoint exists
if 'lsst-lsp-int' in os.environ['EXTERNAL_INSTANCE_URL']:
    analyze_url = urljoin(token_dict['iss'], '/auth/analyze')
    r = requests.post(analyze_url, data={'token': token})
    data = r.json()
    print(json.dumps(data, indent=4))
else:
    logging.info('Not on a platform that supports the analyze endpoint.')